In [6]:
import os
import requests
import zipfile
import pandas as pd

In [7]:
diretorio_principal = os.getcwd()
subdiretorio_dados = os.path.join(diretorio_principal, "Dados")

# Criar o diretório se não existir
os.makedirs(subdiretorio_dados, exist_ok=True)

In [8]:
anos = range(2008, 2024)
formato = "csv"
url_base = "http://www.camara.leg.br/cotas/"

for ano in anos:
    download = requests.get(url_base + f"Ano-{ano}.{formato}.zip")
    caminho_zip = os.path.join(subdiretorio_dados, f"Ano-{ano}.{formato}.zip")

    with open(caminho_zip, "wb") as file:
        file.write(download.content)

    # Extrair o conteúdo do arquivo ZIP
    with zipfile.ZipFile(caminho_zip, 'r') as zip_ref:
        zip_ref.extractall(subdiretorio_dados)

    # # Remover o arquivo ZIP após extração (opcional)
    # os.remove(caminho_zip)

In [9]:
# Voltar ao diretório principal
os.chdir(diretorio_principal)

# Criar um DataFrame com os dados extraídos
despesas_totais = []

for arquivo_csv in os.listdir(subdiretorio_dados):
    if arquivo_csv.endswith('.csv') and not arquivo_csv.startswith("."):
        caminho_csv = os.path.join(subdiretorio_dados, arquivo_csv)
        despesas = pd.read_csv(caminho_csv, sep=";", encoding='UTF-8', low_memory=False)
        despesas_totais.append(despesas)

# Concatenar todos os DataFrames em um único DataFrame
despesas_completo = pd.concat(despesas_totais, ignore_index=True)

In [10]:
despesas_completo

,txNomeParlamentar,cpf,ideCadastro,nuCarteiraParlamentar,nuLegislatura,sgUF,sgPartido,codLegislatura,numSubCota,txtDescricao,...,numParcela,txtPassageiro,txtTrecho,numLote,numRessarcimento,datPagamentoRestituicao,vlrRestituicao,nuDeputadoId,ideDocumento,urlDocumento
0,JOSEPH BANDEIRA,7.251603e+09,4929.0,565.0,2007,BA,PT,53,2,"LOCOMOÇÃO, ALIMENTAÇÃO E HOSPEDAGEM",...,0,NaN,NaN,405260,NaN,NaN,NaN,1970,1612380,NaN
1,JOSEPH BANDEIRA,7.251603e+09,4929.0,565.0,2007,BA,PT,53,2,"LOCOMOÇÃO, ALIMENTAÇÃO E HOSPEDAGEM",...,0,NaN,NaN,387830,NaN,NaN,NaN,1970,1561354,NaN
2,SILVIO COSTA,1.048139e+10,4930.0,160.0,2007,PE,PTB,53,10,TELEFONIA,...,0,NaN,NaN,0,0.0,NaN,NaN,1973,0,NaN
3,ALDO REBELO,1.641215e+10,73428.0,331.0,2007,SP,PCdoB,53,10,TELEFONIA,...,0,NaN,NaN,0,0.0,NaN,NaN,764,0,NaN
4,ANTONIO CARLOS PANNUNZIO,1.895236e+10,73431.0,334.0,2007,SP,PSDB,53,10,TELEFONIA,...,0,NaN,NaN,0,0.0,NaN,NaN,961,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4666855,Dr Fabio Rueda,2.539052e+09,227401.0,569.0,2023,AC,UNIÃO,57,998,PASSAGEM AÉREA - SIGEPA,...,0,FABIO GONCALVES DE RUEDA,BSB/GRU/REC,0,0.0,NaN,NaN,3755,218349,NaN
4666856,Douglas Viegas,2.551906e+10,227433.0,571.0,2023,SP,UNIÃO,57,5,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,...,0,NaN,NaN,2002507,NaN,NaN,NaN,3756,7669098,https://www.camara.leg.br/cota-parlamentar/doc...
4666857,Douglas Viegas,2.551906e+10,227433.0,571.0,2023,SP,UNIÃO,57,10,TELEFONIA,...,0,NaN,NaN,0,0.0,NaN,NaN,3756,0,NaN
4666858,Douglas Viegas,2.551906e+10,227433.0,571.0,2023,SP,UNIÃO,57,120,LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,...,0,NaN,NaN,2003390,NaN,NaN,NaN,3756,7670764,https://www.camara.leg.br/cota-parlamentar/doc...
